## Scrape Zillow Housing Data
- Get all the housing information for renting location around UCSD, i.e. in La Jolla 

Part 1: From General Overall Page
- bs4 + request
- Data Features: 
    1. Price
    2. Address
    3. beds, baths, sqft 

Part 2: From Individual Listing Page  
- Features: 
    1. More Info: Pets, Amenities, etc
    2. Rent ZEstimate History (User Table View)

### Part 1

In [53]:
import requests 
from bs4 import BeautifulSoup as soup

In [55]:
# sample url for La Jolla renting list
url = "https://www.zillow.com/la-jolla-san-diego-ca/rentals" # url refers to the location 

In [56]:
# taken header from browser http request
header = {'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/95.0.4638.69 Safari/537.36 Edg/95.0.1020.53',
    'refer': url} 

In [57]:
# get the website 
html = requests.get(url, headers=header) # don't to run it too much!!!
print(html.status_code)

200


In [58]:
# save it for later experiment
import os
if os.path.exists('zillow_la_jolla.txt'): os.remove('zillow_la_jolla.txt')
with open('zillow_la_jolla.txt', 'w') as file:
    file.write(html.text)

In [59]:
# function to read HTML file back to string and parse it using bs4
def parse_file(fp):
    with open(fp, mode='r') as file:
        html_txt = file.readlines()
    return soup(''.join(html_txt), 'html.parser')
html_txt = parse_file('zillow_la_jolla.txt')

In [60]:
# extract price, and detail info about the house
import re

info=[]
for house in html_txt.findAll('div', {'class':'list-card-heading'}):
    if len(house.text)==0: # if there is nothing (usually happens at the end)
        print('This is the end of the page')
        continue 
    ins={}
    price=house.find('div', {'class': 'list-card-price'}).text
    detail=house.find('ul', {'class': 'list-card-details'}).text
    pattern=r"([1-9] bd)[s]*([1-9] ba)([0-9,\-]+ sqft)-(.*)"
    match=re.match(pattern, detail)
    for index, cat in enumerate(['bds','ba','sqft','house-type']): 
        if match!=None: 
            ins[cat]=match.group(index+1)
        else:
            print("Reach the End")
    info.append(ins)

Reach the End
Reach the End
Reach the End
Reach the End
Reach the End
Reach the End
Reach the End
Reach the End
Reach the End
Reach the End
Reach the End
Reach the End
Reach the End
Reach the End
Reach the End
Reach the End
Reach the End
Reach the End
Reach the End
Reach the End
This is the end of the page


In [61]:
# extract the address of the house 
link_addr = [] # format: (link, addr)
for addr in html_txt.findAll('div', {'class': 'list-card-info'}): 
    ad=addr.find('address', {'class': 'list-card-addr'})
    link=addr.find('a', {'class': 'list-card-link'})
    if ad and link: 
        link_addr.append((link.get('href'), ad.text))
link_addr

[('/b/solazzo-apartments-homes-la-jolla-ca-5Xk5YK/',
  'Solazzo Apartments Homes | 8506 Villa La Jolla Dr, La Jolla, CA'),
 ('/b/ocean-house-on-prospect-apartment-homes-la-jolla-ca-5Xkm98/',
  'Ocean House on Prospect Apartment Homes | 400 Prospect St, La Jolla, CA'),
 ('https://www.zillow.com/homedetails/8446-Via-Sonoma-UNIT-94-La-Jolla-CA-92037/16840698_zpid/',
  '8446 Via Sonoma UNIT 94, La Jolla, CA 92037'),
 ('https://www.zillow.com/homedetails/8524-Via-Mallorca-La-Jolla-Ca-Office-1-BEDROOM-La-Jolla-CA-92037/2067481695_zpid/',
  '8524 Via Mallorca La Jolla Ca Office #1-BEDROOM, La Jolla, CA 92037'),
 ('https://www.zillow.com/homedetails/3161-Morning-Way-La-Jolla-CA-92037/16835830_zpid/',
  '3161 Morning Way, La Jolla, CA 92037'),
 ('https://www.zillow.com/homedetails/356-Playa-Del-Norte-St-La-Jolla-CA-92037/2069746535_zpid/',
  '356 Playa Del Norte St, La Jolla, CA 92037'),
 ('https://www.zillow.com/homedetails/1804-Caminito-Ascua-La-Jolla-CA-92037/16857254_zpid/',
  '1804 Caminit

In [9]:
len(info), len(link_addr)

(9, 9)

In [10]:
# store data in a dataframe 
import pandas as pd 
import numpy as np
combined = []
for i,j in zip(info, link_addr): 
    if len(i)==0 or len(j)==0: 
        continue
    i.update(dict(zip(['link', 'address'], j)))
    combined.append(i)

In [11]:
df = pd.DataFrame(combined)
# remove unit
df['bds'] = df['bds'].str.replace('[bd,]', '', regex=True)
df['ba'] = df['ba'].str.replace('[ba,]', '', regex=True)
df['sqft'] = df['sqft'].str.replace('[sqft,\-]', '', regex=True)
df = df.replace(' ', 0) # should prob leave it as nan
df.astype({'bds': 'int32',
            'ba': 'int32', 
            'sqft': 'int32'})
df

,bds,ba,sqft,house-type,link,address
0,3,3,1992,Coming soon,https://www.zillow.com/homedetails/8619-Via-Ma...,"8619 Via Mallorca UNIT C, La Jolla, CA 92037"
1,4,3,2206,Coming soon,https://www.zillow.com/homedetails/8845-Robin-...,"8845 Robin Hood Ln, La Jolla, CA 92037"
2,5,3,2772,House for sale,https://www.zillow.com/homedetails/8269-Sugarm...,"8269 Sugarman Dr, La Jolla, CA 92037"
3,3,3,1799,House for sale,https://www.zillow.com/homedetails/253-Rosemon...,"253 Rosemont St, La Jolla, CA 92037"
4,4,3,1861,Multi-family home for sale,https://www.zillow.com/homedetails/7256-7258-L...,"7256-7258 La Jolla Blvd, La Jolla, CA 92037"
5,3,3,2284,House for sale,https://www.zillow.com/homedetails/5646-Chelse...,"5646 Chelsea Ave, La Jolla, CA 92037"
6,2,2,992,Condo for sale,https://www.zillow.com/homedetails/457-Coast-B...,"457 Coast Blvd UNIT 403, La Jolla, CA 92037"
7,1,1,668,Condo for sale,https://www.zillow.com/homedetails/860-Turquoi...,"860 Turquoise St UNIT 224, San Diego, CA 92109"


In [39]:
# TODO: Automate multipage scraping 
# TODO: Increase search range to include more housing 
# TODO: Filter the rent data

In [66]:
# Using Dataox's approach 
import numpy as np 
import pandas as pd 
import re 
import lxml 
from lxml.html.soupparser import fromstring
import numbers 
import requests 
from bs4 import BeautifulSoup as soup

In [67]:
request_headers = {
    'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webq,image/apng,*/*;1=0.8', 
    'accept-encoding': 'gzip, deflate, br',
    'accept-language': 'en-US,en;q=0.8',
    'upgrade-insecure-requests': '1', 
    'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/95.0.4638.69 Safari/537.36 Edg/95.0.1020.53',
}

url = 'https://www.zillow.com/la-jolla-san-diego-ca/rentals'
with requests.Session() as session: 
    response = session.get(url, headers=request_headers)

In [71]:
soup = soup(response.content, 'html.parser')

In [78]:
addresses = []
for i in soup:
    address = soup.find_all(class_='list-card-addr')
    addresses.append(address)
print(addresses[0])
print(addresses[1])

[<address class="list-card-addr">Solazzo Apartments Homes | 8506 Villa La Jolla Dr, La Jolla, CA</address>, <address class="list-card-addr">2420 Torrey Pines Rd UNIT B303, La Jolla, CA 92037</address>, <address class="list-card-addr">Ocean House on Prospect Apartment Homes | 400 Prospect St, La Jolla, CA</address>, <address class="list-card-addr">8446 Via Sonoma UNIT 94, La Jolla, CA 92037</address>, <address class="list-card-addr">3161 Morning Way, La Jolla, CA 92037</address>, <address class="list-card-addr">356 Playa Del Norte St, La Jolla, CA 92037</address>, <address class="list-card-addr">1804 Caminito Ascua, La Jolla, CA 92037</address>, <address class="list-card-addr">8529 Villa La Jolla Dr APT E, La Jolla, CA 92037</address>, <address class="list-card-addr">3205 Via Alicante, La Jolla, CA 92037</address>]
[<address class="list-card-addr">Solazzo Apartments Homes | 8506 Villa La Jolla Dr, La Jolla, CA</address>, <address class="list-card-addr">2420 Torrey Pines Rd UNIT B303, La

### Part 2 

In [17]:
df.iloc[0]['link']

'https://www.zillow.com/homedetails/8966-Cliffridge-Ave-La-Jolla-CA-92037/16835000_zpid/'

In [120]:
# get detail specifications from specific rent pages 
# user the first link as an example 
ex_link = df.iloc[0]['link']

# get the website 
page = requests.get(ex_link, headers=header) # don't to run it too much!!!
print(page.status_code)

# save it for later experiment
if os.path.exists('page_example.txt'): os.remove('page_example.txt')
with open('page_example.txt', 'w') as file:
    file.write(page.text)

200


In [5]:
page_txt = parse_file('page_example.txt')

The page elements has dynamic class ids which makes it harder to parse

In [38]:
import re

text = page_txt.text
re.match(r'(Overview)+', text)

### Updated: Use Selenium to simulate scrolling in order to load all the elements

In [1]:
from msedge.selenium_tools import Edge
from msedge.selenium_tools import EdgeOptions
from selenium.common.exceptions import NoSuchElementException
import time
import os
import csv
import requests
from bs4 import BeautifulSoup as soup
import pandas as pd
import re

In [7]:
# initializes the webscraper (function __init__)
options = EdgeOptions()
options.use_chromium = True
# need to find ways to dynamically implement a proxy
# proxy_ip = "103.92.114.2:80" # from freeproxy.com
# options.add_argument("--proxy-server=%s" % proxy_ip)
options.add_argument('--ignore-certificate-errors')
options.add_argument('--incognito')
options.add_argument('start-maximized')
# options.add_argument('--headless') # use headless if don't want a window to pop up
driver = Edge(options = options)

In [8]:
# open target page (might fail, gotta be put into a try block in actual code) (function __init__)
url = "https://www.zillow.com/la-jolla-san-diego-ca/rentals"
# header = {'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/95.0.4638.69 Safari/537.36 Edg/95.0.1020.53', 'refer': url} # not necessary
driver.get(url)
time.sleep(3)

In [9]:
# incrementally scroll to the bottom of the page then extract the page html (function load_page)
scroll_height = driver.find_element_by_id("grid-search-results").size['height']
current, step = 0, 200
while current<scroll_height:
    current += step
    driver.execute_script(f"document.getElementById('search-page-list-container').scrollTo(0, {current})") # 600 per 3 rows of listings
    time.sleep(1)

In [17]:
# save the html source from all the listings (function extract_listings)
listings = driver.find_elements_by_css_selector(".list-card.list-card-additional-attribution.list-card_not-saved")
listings_html = [l.get_attribute('innerHTML') for l in listings]

SyntaxError: unexpected EOF while parsing (Temp/ipykernel_18208/359194341.py, line 7)

In [ ]:
# after scraping a page of data, switch pages (function turn_page, should be embedded in extract_listing)
while True:
    next = driver.find_elements_by_xpath('*[@id="grid-search-results"]/div[3]/nav/ul/li[5]/a')
    if next.is_enabled():
        next.click()
    else:
        break

In [2]:
# temporary block that save html source into a file for later prototyping
def save_listings_html(listings_html, filename="listings_html.txt"):
    if os.path.exists(filename):
        os.remove(filename)
    with open(filename, 'a') as file:
        for lst in listings_html:
            file.write(lst)
            file.write('\n')
def read_listings_html(filename="listings_html.txt"):
    if not os.path.exists(filename):
        return "file doesn't exist"
    with open(filename, 'r') as file:
        lines = file.readlines()
        return lines

In [ ]:
# ran the saving function
save_listings_html(listings_html)

In [3]:
# read the stored listing html source
read_listings = read_listings_html()

In [64]:
# use beautifulsoup to parse the html source (function extract_listings)
def parse_singel_listing(source):
    parsed = soup(source, "html.parser")
    # get the link
    link_header = "https://www.zillow.com"
    link = parsed.find('a', {'class': 'list-card-link'}).get('href')
    flag=False # flag for which pattern structure to use for parsing
    if not link.startswith(link_header, 0, len(link_header)):
        flag=True
        link = link_header + link
    # get basic info
    addr=parsed.find('address', {"class": 'list-card-addr'}).text
    price=parsed.find('div', {'class': 'list-card-price'}).text
    detail=parsed.find('ul', {'class': 'list-card-details'}).text
    if not flag:
        pat=r"([1-9] bd)[s]*([1-9\.]+ ba)([0-9,\\-]+ sqft)- (.*)"
        match=re.match(pat, detail)
        ins={'addr': [addr], 'price': [price], 'link': [link]}
        for index, cat in enumerate(['bds','ba','sqft','type']):
            if match!=None:
                ins[cat]=match.group(index+1).split()[0]
        return pd.DataFrame.from_dict(ins)
    else:
        if len(detail)==0:
            return pd.DataFrame.from_dict({'addr': [addr], 'price': [price.split()[0]], 'bds': [price.split()[1]], 'link': [link]})
        temp_price = [i.split()[0] for i in [price, detail]]
        temp_bds = [i.split()[1] for i in [price, detail]] # include 'studio' as an option
        ins={'addr': [addr]*len(temp_price), 'price': temp_price, 'bds': temp_bds, 'link': [link]*len(temp_price)}
        return pd.DataFrame.from_dict(ins)

In [67]:
# save the parsed data into a csv file (function saved_extracted_data)
df = pd.concat([parse_singel_listing(i) for i in read_listings]).reset_index(drop=True)
# df.to_csv('listing.csv')

In [66]:
df

,addr,price,bds,link,ba,sqft,type
0,Solazzo Apartments Homes | 8506 Villa La Jolla...,"$2,830+",1,https://www.zillow.com/b/solazzo-apartments-ho...,NaN,NaN,NaN
1,"8039 Caminito Mallorca, La Jolla, CA 92037","$3,200/mo",3,https://www.zillow.com/homedetails/8039-Camini...,2.5,"1,467",Townhouse
2,"Prospect | 315 Prospect St, La Jolla, CA","$3,050+",1,https://www.zillow.com/b/prospect-la-jolla-ca-...,NaN,NaN,NaN
3,"Prospect | 315 Prospect St, La Jolla, CA","$4,195+",2,https://www.zillow.com/b/prospect-la-jolla-ca-...,NaN,NaN,NaN
4,"607 Westbourne St, La Jolla, CA 92037","$6,500/mo",3,https://www.zillow.com/homedetails/607-Westbou...,3,"2,012",Townhouse
5,"8368 Via Sonoma, La Jolla, CA","$2,300+",1,https://www.zillow.com/b/la-jolla-park-villas-...,NaN,NaN,NaN
6,Ocean House on Prospect Apartment Homes | 400 ...,"$4,990+",1,https://www.zillow.com/b/ocean-house-on-prospe...,NaN,NaN,NaN
7,Ocean House on Prospect Apartment Homes | 400 ...,"$10,114+",2,https://www.zillow.com/b/ocean-house-on-prospe...,NaN,NaN,NaN
8,"Village Square | 8683 Via Mallorca, La Jolla, CA","$2,425+",1,https://www.zillow.com/b/village-square-la-jol...,NaN,NaN,NaN
9,"1141 SILVERADO ST. | 1141 Silverado St, La Jol...","$1,750+",Studio,https://www.zillow.com/b/1141-silverado-st.-la...,NaN,NaN,NaN
